In [ ]:
import os
import sys

sys.path.append(os.path.abspath("src"))

import logging
import sys

import hydra
import mlflow
import numpy as np
import pandas as pd
import torch
from joblib import Memory
from omegaconf import DictConfig, OmegaConf

from evaluators import torchFastTextEvaluator
from framework_classes import (
    DATA_GETTER,
    DATASETS,
    LOSSES,
    MODELS,
    MODULES,
    OPTIMIZERS,
    PREPROCESSORS,
    SCHEDULERS,
    TOKENIZERS,
    TRAINERS,
)
from models import FastTextWrapper
from utils.data import get_df_naf, get_file_system, get_processed_data, get_test_data, get_Y
from utils.mappings import mappings
from utils.mlflow import create_or_restore_experiment
from utils.validation_viz import calibration_curve, confidence_histogram, sort_and_get_pred

%load_ext autoreload
%autoreload 2

In [ ]:
revision = "NAF2008"
model_class = "torchFastText"
start_month = 1
start_year = 2018
text_feature = "libelle"
textual_features_1 = "NAT_LIB"
textual_features_2 = "AGRI"
categorical_features_1 = "TYP"
categorical_features_2 = "NAT"
categorical_features_3 = "SRF"
categorical_features_4 = "CJ"
categorical_features_5 = "CRT"

In [ ]:
cfg_dict = {"data": 
                {"sirene":"sirene_4", 
                "start_month": start_month, 
                "start_year": start_year, 
                "revision": revision,
                "text_feature": text_feature,
                "textual_features" : [textual_features_1, textual_features_2],
                "categorical_features" : [categorical_features_1, categorical_features_2, categorical_features_3, categorical_features_4, categorical_features_5],}, 
                
            "model":{"name": "fastText",
                    "preprocessor": "fastText"}}
cfg_dict_data = cfg_dict["data"]
df_naf = get_df_naf(revision=cfg_dict_data["revision"])
Y = get_Y(revision=revision)
df_test_ls= get_test_data(**cfg_dict["data"], y=Y)

In [ ]:
import mlflow

logged_model = 'runs:/65bc7a269ea145248476b8c976090784/default'
mlflow.set_tracking_uri("https://projet-ape-mlflow.user.lab.sspcloud.fr/")
# Load model as a PyFuncModel.
fasttext = mlflow.pyfunc.load_model(logged_model)

In [ ]:
fasttext.predict(df_test_ls)

In [ ]:
import mlflow
import pandas as pd

# Set the experiment ID or name
mlflow.set_tracking_uri("https://projet-ape-mlflow.user.lab.sspcloud.fr/")
experiment_name = "model_comparison_s4"
mlflow.set_experiment(experiment_name)

# Retrieve all runs for the experiment
experiment = mlflow.get_experiment_by_name(experiment_name)
runs_df = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# Display the metrics
print(runs_df.head())

In [ ]:
metrics_df = runs_df.filter(like="metrics.")
params_df = runs_df.filter(like="params.")
display(metrics_df)
display(params_df)

In [ ]:
params_df.filter(like="num_trainable_parameters")